In [1]:
#pip install -U sckit-learn

In [2]:
pip install psycopg2-binary

  Using cached psycopg2_binary-2.9.3-cp37-cp37m-win_amd64.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\jonwm\anaconda3\envs\PythonData\envs\mlenv\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install graphviz

     ---------------------------------------- 47.0/47.0 KB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\jonwm\anaconda3\envs\PythonData\envs\mlenv\python.exe -m pip install --upgrade pip' command.


In [5]:
import pandas as pd
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import requests
import os
import psycopg2


from sqlalchemy import create_engine
from config import db_password

In [7]:
#Create our connection
connection_dic = {
    "host":"localhost",
    "database":"FinalProject",
    "user":"postgres",
    "password":db_password
}
def connect(connection_dic):
    #Connecting to the postgresql db server
    conn = None
    try:
        #connect to the server
        print('Connecting to the Final Project database...')
        conn = psycopg2.connect(**connection_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print("Connection Successful!")
    return conn

In [ ]:
def getdata (conn, select_query, column_names):
    #create our dataframe from db
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception,psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [ ]:
#Grab the courses_df
conn = connect(connection_dic)
driver_cols = ["position","name","rarity","special_skill"]
courses_cols = ["name","is_reverse","is_trick"]
level1_df = ["driver_id_1","course_name","special_skill_1","rarity_1","win_loss_1","driver_id_2","rarity_2","special_skill_2"
            ,"driver_id_3","special_skill_3","rarity_3"]
final_mk_cols = ["course_name","name","special_skill","rarity","win_loss"]

# Grab the drivers data for our connection
sql_query = "select * from drivers"
drivers_df = getdata(conn, sql_query, column_names)


#Grab the courses data
sql_query = ("select * from courses")
courses_df = getdata(conn, sql_query, courses_cols)

# grab the level 1 driver information data
sql_query = ("select * from level1")
level1_df = getdata(conn, sql_query, level1_cols)

# grab the finalized data from our db
sql_query = ("select * from (name of db)")
df = getdata(conn, sql_query, final_mk_cols)


In [6]:
#Importing Dataset
data = 'Resources/final_mariokart_data.csv'
df = pd.read_csv (data)
df

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/final_mariokart_data.csv'

In [150]:
#Remove driver_id column
df.drop('driver_id', axis=1, inplace=True)
df

,course_name,name,special_skill,rarity,win_loss
0,DS Waluigi Pinball,Baby Mario,Normal,Boomerang Flower,1
1,SNES Mario Circuit 1T,Baby Mario,Normal,Boomerang Flower,1
2,3DS Cheep Cheep Lagoon,Baby Peach,Normal,Bubble,1
3,3DS Daisy Hills R/T,Baby Peach,Normal,Bubble,1
4,3DS Daisy Hills T,Baby Daisy,Normal,Bubble,1
...,...,...,...,...,...
535,3DS Neo Bowser City T,Waluigi (Bus Driver),High-End,Triple Bananas,0
536,Vancouver Velocity R,Wario (Hiker),High-End,Bob-omb Cannon,0
537,GBA Bowser's Castle 2T,Dry Bones (Gold),High-End,Coin Box,0
538,DS Airship Fortress R,Dry Bowser (Gold),High-End,Bob-omb Cannon,0


In [133]:
#Check the datatypes
df.dtypes

course_name      object
name             object
special_skill    object
rarity           object
win_loss          int64
dtype: object

In [142]:
#Set win_loss as target
inputs = df.drop('win_loss',axis='columns')
target = df['win_loss']

In [143]:
#Add Encoder Library
from sklearn.preprocessing import LabelEncoder

In [144]:
#Identify columns to encode
le_course_name = LabelEncoder()
le_name = LabelEncoder()
le_special_skill = LabelEncoder()
le_rarity = LabelEncoder()

In [156]:
#Encode columns, associate a number to text
inputs['course_name_new'] = le_course_name.fit_transform(inputs['course_name'])
inputs['name_new'] = le_name.fit_transform(inputs['name'])
inputs['special_skill_new'] = le_name.fit_transform(inputs['special_skill'])
inputs['rarity_new'] = le_name.fit_transform(inputs['rarity'])
inputs

,course_name,name,special_skill,rarity,course_name_new,name_new,special_skill_new,rarity_new
0,DS Waluigi Pinball,Baby Mario,Normal,Boomerang Flower,46,2,1,3
1,SNES Mario Circuit 1T,Baby Mario,Normal,Boomerang Flower,147,2,1,3
2,3DS Cheep Cheep Lagoon,Baby Peach,Normal,Bubble,0,3,1,5
3,3DS Daisy Hills R/T,Baby Peach,Normal,Bubble,6,3,1,5
4,3DS Daisy Hills T,Baby Daisy,Normal,Bubble,7,0,1,5
...,...,...,...,...,...,...,...,...
535,3DS Neo Bowser City T,Waluigi (Bus Driver),High-End,Triple Bananas,14,95,0,18
536,Vancouver Velocity R,Wario (Hiker),High-End,Bob-omb Cannon,172,97,0,2
537,GBA Bowser's Castle 2T,Dry Bones (Gold),High-End,Coin Box,56,29,0,6
538,DS Airship Fortress R,Dry Bowser (Gold),High-End,Bob-omb Cannon,36,31,0,2


In [146]:
#Remove course_name, name, special_skill & rarity
inputs_new = inputs.drop(['course_name', 'name', 'special_skill', 'rarity'],axis='columns')
inputs_new

,course_name_new,name_new,special_skill_new,rarity_new
0,46,2,1,3
1,147,2,1,3
2,0,3,1,5
3,6,3,1,5
4,7,0,1,5
...,...,...,...,...
535,14,95,0,18
536,172,97,0,2
537,56,29,0,6
538,36,31,0,2


In [160]:
#Add combination of numbers to predict win=1 or loss=0
model.predict([[46,3,1,5]])

array([1])

In [123]:
pip install six

Note: you may need to restart the kernel to use updated packages.


In [124]:
from sklearn import tree

In [140]:
# Importing required packages for visualization
from IPython.display import Image  
from six import StringIO
from sklearn.tree import export_graphviz
import pydotplus, graphviz

In [141]:
# plotting tree with max_depth=3
dot_data = StringIO()  

export_graphviz(inputs_new, out_file=dot_data, filled=True, rounded=True,
                feature_names=['course_name_new', "name_new", "special_skill_new", "rarity_new"], 
                class_names=['win_loss', "Win"])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.tree_png())

TypeError:      course_name_new  name_new  special_skill_new  rarity_new
0                 46         2                  1           3
1                147         2                  1           3
2                  0         3                  1           5
3                  6         3                  1           5
4                  7         0                  1           5
..               ...       ...                ...         ...
535               14        95                  0          18
536              172        97                  0           2
537               56        29                  0           6
538               36        31                  0           2
539               22        64                  0          21

[540 rows x 4 columns] is not an estimator instance.

In [96]:
model = tree.DecisionTreeClassifier()

In [97]:
model.fit(inputs_new,target)

DecisionTreeClassifier()

In [98]:
model.score(inputs_new,target)

1.0

In [108]:
import warnings 
warnings.filterwarnings("ignore")